# Tutorials i TensorFlow fra Kieth Downing
## Sett 2, Autoencoder.

Dette eksempelet er et neuralt nett som baserer seg på bits. Den skal få inn bits, så gjennom et hemmlig lag bestående av færre noder enn inputverdier skal den finne tilbake til bits som var i input.

Autoencoder er her laget som en egen klasse. Denne klassen kan bygge det neurale nettet med gitte parametere som spesifiserer hvordan nettet skal se ut. 

Keith downings forklarende tekst:

We can extend the basic approach in tfex8 (tutor1.py) to handle 
1. A 3-layered neural network
2. A collection of cases to be learned.  

This is a specialized neural network designed to solve one type of classification problem: converting an input string, through a single hidden layer, to a copy of itself on the output end.


# Oppbygging av nettet:

In [12]:
## HENTET FRA autoencoder():
def __init__(self,nh=3,lr=.1):
    self.cases = TFT.gen_all_one_hot_cases(2**nh)
    self.learning_rate = lr
    self.num_hiddens = nh
    self.build_neural_network(nh)

Her brukes det to variabler som input for metoden. Disse to skal spesifisere nettet. 

```python
	nh = 3   # number of hidden nodes in the hidden layer
	lr = 0.2 # learning rate used with gradient decent.
```

Casene hentes fra en genereringsmetode laget av downing. Denne lager bare bit-strenger. Altså sekvenser med bits. Her kommer sekvensene i lengde 2*nh. De hver sekvens er altså dobbelt så lage som antall hidden nodes.

Hvis det er tre noder i det hemmelige laget, kommer input i form av:
```
	011001
```

## BYGGING AV DET NEURALE NETTET:
Under kommer forklaring av koden i metoden "build_neural_network()".

In [13]:
def build_neural_network(self,nh):
    ios = 2**nh  # ios = input- and output-layer size
    self.w1 = tf.Variable(np.random.uniform(-.1,.1,size=(ios,nh)),name='Weights-1')  # first weight array
    self.w2 = tf.Variable(np.random.uniform(-.1,.1,size=(nh,ios)),name='Weights-2') # second weight array
    self.b1 = tf.Variable(np.random.uniform(-.1,.1,size=nh),name='Bias-1')  # First bias vector
    self.b2 = tf.Variable(np.random.uniform(-.1,.1,size=ios),name='Bias-2')  # Second bias vector
    self.input = tf.placeholder(tf.float64,shape=(1,ios),name='Input')
    self.target = tf.placeholder(tf.float64,shape=(1,ios),name='Target')
    self.hidden = tf.sigmoid(tf.matmul(self.input,self.w1) + self.b1,name="Hiddens")
    self.output = tf.sigmoid(tf.matmul(self.hidden,self.w2) + self.b2, name = "Outputs")
    self.error = tf.reduce_mean(tf.square(self.target - self.output),name='MSE')
    self.predictor = self.output  # Simple prediction runs will request the value of outputs
    # Defining the training operator
    optimizer = tf.train.GradientDescentOptimizer(self.learning_rate)
    self.trainer = optimizer.minimize(self.error,name='Backprop')

Det bortgjemte laget skal alltid være halve størrelsen av input - og output lagene. 
```
ios = 2**nh
```

Han definerer 2 forskjellige sett med vekter her. Det må to sett til fordi det er et hidden-layer mellom input og output. All kommunikasjon skjer gjennom vektmatrisene. Vektmatrisene kobler sammen de forskjellige lagene. Det er viktig at de forskjellige vektmatrisene er riktig størrelse. 

Input er 5 i bredde, mens hidden layer er 3. Dette betyr at vektmatrise 1 (w1) har dimensjonene 5x3. 

Hidden layer er bredde 3, mens output er lengde 5. Dette betyr at vektmatrise 2 (w2) må være på dimensjonene 3x5

```
	self.w1 = tf.Variable(np.random.uniform(-.1,.1,size=(ios,nh)),name='Weights-1')  # first weight array
	self.w2 = tf.Variable(np.random.uniform(-.1,.1,size=(nh,ios)),name='Weights-2') # second weight array
   
```

![modell](https://github.com/MagnusPoppe/Tensorflow-Interface/blob/master/3-node%20Modell.png?raw=true)

Det produserers så to bias. Bias betyr "partiskhet" og er viktig i å "overtale" nettet til å velge riktig. En bias hjelper med å gi en verdi som er foretrukket. 
```
	self.b1 = tf.Variable(np.random.uniform(-.1,.1,size=nh),name='Bias-1')  # First bias vector
	self.b2 = tf.Variable(np.random.uniform(-.1,.1,size=ios),name='Bias-2')  # Second bias vector
```
Selve bias variabelen er like stor som laget den skal tilhøre. Biasen blir koblet inn på laget som en addisjon til matrisemultiplikasjonen. Svaret er addert med bias. Dette gjøres da etter multiplikasjonen med vektene.

![modell](https://github.com/MagnusPoppe/Tensorflow-Interface/blob/master/bias-model.png?raw=true)

Han definerer input og target som tf.placeholder() variabler. Dette er fordi det skal mates inn verdier i disse kolonnene. 
```
	self.input  = tf.placeholder(tf.float64,shape=(1,ios),name='Input')
	self.target = tf.placeholder(tf.float64,shape=(1,ios),name='Target')
```    

- Input er inn-verdiene. Disse er data direkte fra modellen, slik som tensorflow skal tolke dem.
- Target er de ønskede verdiene vi vil se når inputverdiene har gått igjennom nettet. Targetverdien er altså brukt til å sammenlikne med output. Under trening er target brukt til å vise hvor bra loss vi får ut. 

At størrelsen på de to matrisene er like er tilfeldig. Target matrisen skal være tilsvarende størrelse som output laget, siden disse skal sammenliknes. 

Data som blir sendt med i "feed_dict" blir plassert i disse to variablene. De blir matet direkte inn i par. Du vil ha en input-verdi og en ønsket output-verdi eller target-verdi. Begge disse er altså da forhåndsdefinert. 

To lag blir så definert. Først hidden layer. 

```
	self.hidden = tf.sigmoid( tf.matmul(self.input,  self.w1) + self.b1, name = "Hiddens")
	self.output = tf.sigmoid( tf.matmul(self.hidden, self.w2) + self.b2, name = "Outputs")
```     

Både hidden og output bruker sigmoid funksjonen for å trene vektene som ligger i vekt-lagene. Selve verdiene som hidden-layer og output består av er kun operatorer i tensorflow. Det er matrisemultiplikasjon: 
```
	forrige lag * vektmatrisen + bias.
```

Error metoden er i dette tilfellet differansen mellom target og output. Svaret på dette kvadreres. Error funksjonen baserer seg så på å bruke en "reduce" metode. En metode for å gjøre om mange verdier om til en verdi. I dette tilfellet brukes median. Andre metoder kan være f.eks. avg.
```
        self.error = tf.reduce_mean(tf.square(self.target - self.output),name='MSE')
```

Han definerer så treningsalgoritmen, eller optimaliseringsalgoritmen.
```
	optimizer = tf.train.GradientDescentOptimizer(self.learning_rate)
	self.trainer = optimizer.minimize(self.error,name='Backprop') 
```

# Trening og valideringstesting
Hele treningmetoden for autoencoder er i metoden under. Denne metoden inneholder også valideringstesting. 

In [14]:
def do_training(self,epochs=100,test_interval=10,show_interval=50):
        errors = []
        if test_interval: self.avg_vector_distances = []
        self.current_session = sess = TFT.gen_initialized_session()
        step = 0
        for i in range(epochs):
            error = 0
            grabvars = [self.error]
            for c in self.cases:
                feeder = {self.input: [c[0]], self.target: [c[1]]}
                _,grabvals,_ = self.run_one_step([self.trainer],grabvars,step=step,show_interval=show_interval,
                                                 session=sess,feed_dict=feeder)
                error += grabvals[0]
                step += 1
            errors.append(error)
            if (test_interval and i % test_interval == 0):
                self.avg_vector_distances.append(calc_avg_vect_dist(self.do_testing(sess,scatter=False)))
        PLT.figure()
        TFT.simple_plot(errors,xtitle="Epoch",ytitle="Error",title="")
        if test_interval:
            PLT.figure()
            TFT.simple_plot(self.avg_vector_distances,xtitle='Epoch',
                              ytitle='Avg Hidden-Node Vector Distance',title='')

Metoden begynner med å definere verdier han ønsker å lagre. Listen __"Errors"__ er en liste over hvordan loss funksjonen ser ut for hvert "timestep". Listen __"avg_vector_distances()"__ holder informasjon om alle runder med valideringstesting. 

In [15]:
for i in range(epochs):
    error = 0
    grabvars = [self.error]
    for c in self.cases:
        feeder = {self.input: [c[0]], self.target: [c[1]]}
        _,grabvals,_ = self.run_one_step([self.trainer],grabvars,step=step,show_interval=show_interval,
                                         session=sess,feed_dict=feeder)
        error += grabvals[0]
        step += 1
    errors.append(error)
    if (test_interval and i % test_interval == 0):
        self.avg_vector_distances.append(calc_avg_vect_dist(self.do_testing(sess,scatter=False)))

NameError: name 'epochs' is not defined

Den ytre for-løkken over går igjennom epoker. En epoke er en full gjennomgang av datasettet. Her setter han erroren og definerer at han skal ha den ut av session.run() etter metoden har blitt kalt. Videre kommer en ny for løkke.

In [16]:
for c in self.cases:
    feeder = {self.input: [c[0]], self.target: [c[1]]}
    _,grabvals,_ = self.run_one_step([self.trainer],grabvars,step=step,show_interval=show_interval,
                                     session=sess,feed_dict=feeder)
    error += grabvals[0]
    step += 1

NameError: name 'self' is not defined

I den indre for-løkken kjøres hver mini-batch. I dette tilfellet ser vi at han kun har 1 verdi per mini-batch. Dette blir da ét case per runde i løkken. 

Case-verdiene sendes inn i en feeder-dictionary som senere blir brukt med session.run() sin feed_dict. Traineren kjøres en runde med variablene som er blitt oppgitt. Ut kommer de ønskede "grab vals". 

Videre kommer kallet på __run_one_step()__

In [17]:
def run_one_step(self,operators, grabbed_vars=None, dir='probeview',
              session=None, feed_dict=None, step=1, show_interval=1):
    sess = session if session else TFT.gen_initialized_session(dir=dir)

    results = sess.run([operators, grabbed_vars], feed_dict=feed_dict)
    if show_interval and (step % show_interval == 0):
        TFT.show_results(results[1], grabbed_vars, dir)
    return results[0], results[1], sess

Metoden sjekker først om den allerede har en session. Hvis dette ikke er tilfellet, blir en ny session laget. Session.run() blir så kalt med de ønskede verdiene. Resten av koden er for visualisering. Verdiene returneres. 

Legg merke til at session også returneres. Dette er fordi session skal brukes om igjen i neste runde av løkken. 

Tilbake i "do_training()" er vi nå ute av den indre løkken. __Error__ legges til i listen over alle errors. Det følgende som skjer er valideringstestingen. Her kalles "do-testing()" metoden, men kun om det er ønsket at testing skal skje. Resultatet fra testingen lagres i listen __"avg_vector_distances()"__.

# Testing

In [18]:
# This particular testing is ONLY called during training, so it always receives an open session.
def do_testing(self,session=None,scatter=True):
    sess = session if session else self.current_session
    hidden_activations = []
    grabvars = [self.hidden]
    
    for c in self.cases:
        feeder = {self.input: [c[0]]}
        _,grabvals,_ = self.run_one_step([self.predictor],grabvars,session=sess,
                                         feed_dict = feeder,show_interval=None)
        hidden_activations.append(grabvals[0][0])
        
    if scatter:
        PLT.figure()
        vs = hidden_activations if self.num_hiddens > 3 else TFT.pca(hidden_activations,2)
        TFT.simple_scatter_plot(hidden_activations,radius=8)
        
    return hidden_activations

__"do_testing()"__ er veldig lik "do-training". Denne er litt enklere. I testing metoden er det det gjemte laget vi ser på. Vi kjører kun operatoren __"self.predictor"__. Dette er da en kopi av verdien til output. Testene kjøres for 1 epoke, altså alle casene, 1 gang. 

"if scatter" delen er kun grafikk. Ikke vikitg.